In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np #대수학
import pandas as pd #전처리

import seaborn as sns #시각화
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model

plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

import warnings
%matplotlib inline

import configparser
from sklearn.preprocessing import MinMaxScaler

import re
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import datetime as dt

import os # 경로

import numpy as np
import sys
import random 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

import configparser
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.layers import Dropout
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [6]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [7]:
import IPython

### covid 포함

In [8]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gdp_re/training_data_0.csv')
# data = data.drop(['Unnamed: 0'], axis=1)
data.tail()

,Y,YY1,YY2,YY3,MONTH,YEAR,QRT,TIME,STATE,X1_L2,...,X16_L0,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3
82,1.1,['1.1' '1.7'],['1.1' '1.7' '0.8'],['1.1' '1.7' '0.8' '0.3'],10,2020,2020Q4,202010,0,-0.042056,...,-0.026092,0.000739,0.034143,0.066897,0.203978,-0.061557,0.084760,0.092080,-0.084401,0.086671
83,1.7,['1.7' '0.8'],['1.7' '0.8' '0.3'],['1.7' '0.8' '0.3' '1.2'],1,2021,2021Q1,202101,0,0.028918,...,0.035755,0.108874,0.142995,-0.026092,0.086006,0.034970,-0.060193,0.111819,0.031924,0.003529
84,0.8,['0.8' '0.3'],['0.8' '0.3' '1.2'],['0.8' '0.3' '1.2' 'nan'],4,2021,2021Q2,202104,0,-0.045198,...,0.028235,0.016087,0.012345,0.035755,0.181708,-0.027473,-0.082522,0.172976,-0.046184,0.000645
85,0.3,['0.3' '1.2'],['0.3' '1.2' 'nan'],['0.3' '1.2' 'nan' 'nan'],7,2021,2021Q3,202107,0,-0.008101,...,-0.028623,0.028952,0.017809,0.028235,0.065659,-0.008261,-0.025878,0.040323,-0.053798,0.017722
86,1.2,['1.2' 'nan'],['1.2' 'nan' 'nan'],['1.2' 'nan' 'nan' 'nan'],10,2021,2021Q4,202110,0,-0.028597,...,-0.031980,-0.040775,-0.000952,-0.028623,0.056783,-0.035722,0.014714,0.006634,-0.037503,0.069970


In [9]:
data.iloc[50:52]

,Y,YY1,YY2,YY3,MONTH,YEAR,QRT,TIME,STATE,X1_L2,...,X16_L0,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3
50,0.5,['0.5' '0.9'],['0.5' '0.9' '1.2'],['0.5' '0.9' '1.2' '0.9'],10,2012,2012Q4,201210,0,-0.039153,...,-0.042155,0.047813,0.012290,0.015092,0.087083,-0.051494,-0.009766,0.034979,-0.018330,0.000172
51,0.9,['0.9' '1.2'],['0.9' '1.2' '0.9'],['0.9' '1.2' '0.9' '0.8'],1,2013,2013Q1,201301,0,0.025424,...,-0.017581,0.033188,0.010899,-0.042155,-0.068809,0.021236,0.009105,-0.012413,0.003404,0.020075


In [10]:
data = data[data.iloc[:, 9:].apply(lambda row : sum(pd.isna(row)), axis = 1) == 0]
data = data.reset_index(drop = True)
data

,Y,YY1,YY2,YY3,MONTH,YEAR,QRT,TIME,STATE,X1_L2,...,X16_L0,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3
0,2.5,['2.5' '1.8'],['2.5' '1.8' '2'],['2.5' '1.8' '2' '1.1'],1,2002,2002Q1,200201,0,0.038710,...,0.078377,0.077358,0.197244,0.121185,-0.073605,0.047704,-0.050085,-0.032690,0.008161,-0.017273
1,1.8,['1.8' '2'],['1.8' '2' '1.1'],['1.8' '2' '1.1' '-0.7'],4,2002,2002Q2,200204,0,-0.071885,...,-0.059448,0.092184,0.096141,0.078377,0.221869,-0.060516,-0.010717,0.141282,-0.076678,0.011587
2,2.0,['2' '1.1'],['2' '1.1' '-0.7'],['2' '1.1' '-0.7' '-0.2'],7,2002,2002Q3,200207,0,0.016566,...,-0.033297,-0.067403,-0.054539,-0.059448,-0.074883,0.034205,0.008461,-0.062304,0.011802,0.048913
3,1.1,['1.1' '-0.7'],['1.1' '-0.7' '-0.2'],['1.1' '-0.7' '-0.2' '1.9'],10,2002,2002Q4,200210,0,-0.030120,...,0.019337,-0.122189,0.025641,-0.033297,-0.012388,0.015409,0.072513,0.018233,-0.002239,0.084041
4,-0.7,['-0.7' '-0.2'],['-0.7' '-0.2' '1.9'],['-0.7' '-0.2' '1.9' '2.6'],1,2003,2003Q1,200301,0,0.004367,...,-0.056872,-0.134175,0.099982,0.019337,-0.037133,-0.031773,0.125118,0.027707,0.017755,0.055096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.1,['1.1' '1.7'],['1.1' '1.7' '0.8'],['1.1' '1.7' '0.8' '0.3'],10,2020,2020Q4,202010,0,-0.042056,...,-0.026092,0.000739,0.034143,0.066897,0.203978,-0.061557,0.084760,0.092080,-0.084401,0.086671
76,1.7,['1.7' '0.8'],['1.7' '0.8' '0.3'],['1.7' '0.8' '0.3' '1.2'],1,2021,2021Q1,202101,0,0.028918,...,0.035755,0.108874,0.142995,-0.026092,0.086006,0.034970,-0.060193,0.111819,0.031924,0.003529
77,0.8,['0.8' '0.3'],['0.8' '0.3' '1.2'],['0.8' '0.3' '1.2' 'nan'],4,2021,2021Q2,202104,0,-0.045198,...,0.028235,0.016087,0.012345,0.035755,0.181708,-0.027473,-0.082522,0.172976,-0.046184,0.000645
78,0.3,['0.3' '1.2'],['0.3' '1.2' 'nan'],['0.3' '1.2' 'nan' 'nan'],7,2021,2021Q3,202107,0,-0.008101,...,-0.028623,0.028952,0.017809,0.028235,0.065659,-0.008261,-0.025878,0.040323,-0.053798,0.017722


In [11]:
data['TIME'] = data['TIME'].astype(str)

time_list = []
for i in range(len(data)):
  time_i = data['TIME'][i]
  time = re.sub(r'(\d{4})(\d{2})', '\g<1>-\g<2>', time_i) #정규표현식 'YYYY-mm'
  time_list.append(time)
      
data['TIME'] = time_list
data['datetime'] = pd.to_datetime(data['TIME'],  format="%Y-%m") #datetime변환
data.sort_values('datetime', ignore_index = True, inplace=True)
data.set_index(['YEAR'], inplace=True)

In [12]:
train = data.drop(['YY1','YY2','YY3','MONTH','QRT','TIME','STATE', 'datetime'], axis=1)
train.columns

Index(['Y', 'X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'],
      dtype='object')

In [13]:
# # ##정규화
# # data['ride_passen_log'] = np.log1p(data['ride_passen'])
# # data['takeoff_passen_log'] = np.log1p(data['takeoff_passen'])
# # data.head(13)

# ## 표준화
# from sklearn.preprocessing import StandardScaler
# # train = train.drop(['MONTH', 'YEAR', 'QRT', 'TIME', 'STATE', 'Y1',	'Y2',	'Y3',	'YY1',	'YY2',	'YY3', 'datetime'], axis=1)
# train = StandardScaler().fit_transform(train)
# train

In [14]:
# ## 정규화
# scaler = MinMaxScaler()
# scaler.fit(train)
# train = scaler.transform(train)
# train

In [15]:
df = pd.DataFrame(train, columns=['Y', 'X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'], index=data.index)
# df['YEAR'] = data['YEAR']

In [17]:
def seed_everthing(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everthing(1)

In [19]:
def lstm_model(hp):
  model = Sequential()

  model.add(Embedding(input_dim =52, output_dim = 1))


  hp_units = hp.Int('units', min_value = 26, max_value = 512, step = 52/2)
  hp_dropout = hp.Choice('drop_out', values = [0.1, 0.2, 0.3, 0.4])

  # RNN 첫번째 은닉층 추가
  model.add(LSTM(units = hp_units, activation='tanh', return_sequences = True))
  model.add(Dropout(hp_dropout)) # 노드간의 커넥트를 랜덤하게 끊어줌

  # RNN 두번째 은닉층 추가
  model.add(LSTM(int(hp_units/2), activation='tanh', return_sequences = True))
  model.add(Dropout(hp_dropout))

  # RNN 세번째 은닉층 추가
  model.add(LSTM(int(hp_units/3), activation='tanh', return_sequences = True))
  model.add(Dropout(hp_dropout))

  # RNN 네번째 은닉층 추가
  model.add(LSTM(10, activation='tanh', return_sequences = False))
  model.add(Dropout(hp_dropout))

  # 출력층 추가
  model.add(Dense(units = 1))

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
  # checkpoint = ModelCheckpoint(filepath = f"./model_{i}.h5", monitor='val_loss', save_best_only=True)
  # early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

  # initial_learning_rate = 0.01
  # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps = 100, decay_rate = 0.96, staircase=True)
  # scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

  # hist = model.fit(X_train, y_train, validation_data  = (X_test, y_test), epochs=150,  batch_size =16 ,callbacks=[checkpoint, early_stopping, scheduler])
  # model_file = [fl for fl in os.listdir("/content") if re.findall(f"model_{i}", fl)][0]
  # model = load_model(f"/content/{model_file}")

  return model

In [20]:
tuner = kt.Hyperband(lstm_model,
                     objective = 'val_loss', 
                     max_epochs = 150,
                     factor = 3)

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

In [ ]:
from sklearn.model_selection import train_test_split

df_train = df[df.index < 2015] # 학습데이터

x_train, x_val, y_train, y_val = train_test_split(df_train.drop(["Y"], axis =1), df_train["Y"] ,test_size = 0.25, shuffle = False)
tuner.search(x_train, y_train, epochs = 150, validation_data = (x_val, y_val), callbacks = [early_stopping])

Trial 32 Complete [00h 00m 11s]
val_loss: 0.2321828305721283

Best val_loss So Far: 0.06296921521425247
Total elapsed time: 00h 07m 30s

Search: Running Trial #33

Value             |Best Value So Far |Hyperparameter
234               |234               |units
0.1               |0.4               |drop_out
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)


NameError: ignored

In [ ]:
best_hps.values

{'drop_out': 0.4,
 'tuner/bracket': 4,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units': 234}

In [ ]:
from keras.backend import rnn
sample_size = len(df)
predicted_y = []

df_train = df[df.index < 2015] # 학습데이터
window = len(df_train) # 윈도우 사이즈 설정 (2002~2015)


df_test = df[df.index >= 2015] #검증데이터
forecast_horizon = len(df_test) # 검증데이터 길이

# X_test = df_test.drop(['Y'], axis=1)#.to_numpy() 
# y_test = df_test['Y']#.to_numpy()

start_index = 0

# fit the model and make a forecast
for i in tqdm(range(forecast_horizon)): #검증데이터만큼 돌린다

    # writer = SummaryWriter()

    end_index = start_index + window 
    new_data = df.iloc[start_index : end_index] # df에서 end_index+1

    x_train, x_val, y_train, y_val = train_test_split(new_data.drop(["Y"], axis =1), new_data["Y"] ,test_size = 0.25, shuffle = False)

    # X = new_data.drop(['Y'], axis=1) #학습되는 'X' 
    # y = new_data['Y'] # 학습되는 'Y'

    # gets the trained neural network with given data
    model = rnn_model(x_train, y_train, x_val, y_val) #rnn_moel함수는 hist를 return해줌

    x_new = df.iloc[end_index]
    x_new = pd.DataFrame(x_new).T
    y_pred = model.predict(x_new)
    # y_pred = y_pred[0][0]    

    predicted_y.append(y_pred)

    # predicted_y[end_index+1] = y_hat

    #start_index = start_index + 1 # for rolling-window forecasts

    window = window + 1 # for recursive forecasts
   

Trial 20 Complete [00h 00m 07s]
val_loss: 0.2403663545846939

Best val_loss So Far: 0.06299874931573868
Total elapsed time: 00h 03m 15s

Search: Running Trial #21

Value             |Best Value So Far |Hyperparameter
468               |286               |units
0.4               |0.4               |drop_out
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


  0%|          | 0/28 [03:04<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
predicted_y = [v[0][0] for v in predicted_y]

In [ ]:
df_test["Y"]

YEAR
2015    0.650794
2015    0.587302
2015    0.761905
2015    0.634921
2016    0.571429
2016    0.698413
2016    0.603175
2016    0.619048
2017    0.682540
2017    0.634921
2017    0.761905
2017    0.476190
2018    0.698413
2018    0.619048
2018    0.634921
2018    0.650794
2019    0.492063
2019    0.682540
2019    0.587302
2019    0.730159
2020    0.317460
2020    0.015873
2020    0.873016
2020    0.698413
2021    0.793651
Name: Y, dtype: float64

In [ ]:
# df_test["loss"] = predicted_y - df_test["Y"]
df_test["predicted_y"] = predicted_y

In [ ]:
df_test[["Y", "predicted_y", "loss"]].to_csv("./prediction_result.csv", sep = ",", encoding = "cp949")

In [ ]:
print((np.sum((predicted_y - df_test["Y"]) ** 2) / len(predicted_y)) ** 1/2)

0.014813777979617013


In [ ]:
data = pd.read_csv('training_data_2.csv')



X = data.iloc[:, 9:]

scaler = MinMaxScaler()

scaler.fit(X)

X = scaler.transform(X)

data.iloc[:, 9:] = X



sample_size = len(data)

predicted_y = np.zeros(sample_size)





# 2015 is the starting year of the out-of-sample period



df_pre = data.loc[data['YEAR'] < 2015]

window = len(df_pre)

df_post = data.loc[data['YEAR'] >= 2015]
forecast_horizon = len(df_post)

start_index = 0

for i in range(forecast_horizon):

    

    end_index = start_index + window

    

    new_data = data.iloc[start_index: end_index]

    

    X = new_data.iloc[:, 9:].to_numpy() 

    y = new_data['Y'].to_numpy()

    

    model = train_network(X, y)

  
    x_new = data.iloc[end_index, 9:].to_numpy()
    y_hat = model(x_new, training = False)

    predicted_y[end_index] = y_hat

    

    #start_index = start_index + 1 # for rolling-window forecasts

    window = window + 1 # for recursive forecasts

In [ ]:
 y_vloss = hist.history['loss']
y_loss = hist.history['val_loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
np.sqr(predicted_y_list)

In [ ]:
## 교차검증 시 데이터 분할 시각화 helper function 가져오기
from sklearn.model_selection import TimeSeriesSplit 
n_split = 4
tscv = TimeSeriesSplit(n_splits=n_split)
X = X.set_index(data["datetime"])

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
  print("Fold: {}".format(fold))
  print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
  print("\n")
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  

### covid 미포함

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gdp_re/training_data_0524.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data.tail()

In [ ]:
data = data[data.iloc[:, 9:].apply(lambda row : sum(pd.isna(row)), axis = 1) == 0]
data = data.reset_index(drop = True)
data

In [ ]:
data = data[(data['YEAR']!=2020)&(data['YEAR']!=2021)]
data

In [ ]:
data['TIME'] = data['TIME'].astype(str)

time_list = []
for i in range(len(data)):
  time_i = data['TIME'][i]
  time = re.sub(r'(\d{4})(\d{2})', '\g<1>-\g<2>', time_i) #정규표현식 'YYYY-mm'
  time_list.append(time)
      
data['TIME'] = time_list
data['datetime'] = pd.to_datetime(data['TIME'],  format="%Y-%m") #datetime변환
data.sort_values('datetime', ignore_index = True, inplace=True)

In [ ]:
data.tail()

In [ ]:
train = data.drop(['Y1','Y2','Y3','YY1','YY2','YY3','MONTH','YEAR','QRT','TIME','STATE','datetime'], axis=1)
train.columns
## 다 떄려넣기
## 성능확인
## 너무 다른얟,림ㅇ

In [ ]:
# ##정규화
# data['ride_passen_log'] = np.log1p(data['ride_passen'])
# data['takeoff_passen_log'] = np.log1p(data['takeoff_passen'])
# data.head(13)

## 표준화
from sklearn.preprocessing import StandardScaler
# train = train.drop(['MONTH', 'YEAR', 'QRT', 'TIME', 'STATE', 'Y1',	'Y2',	'Y3',	'YY1',	'YY2',	'YY3', 'datetime'], axis=1)
train = StandardScaler().fit_transform(train)
train

In [ ]:
# ## 정규화
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
train

In [ ]:
train = pd.DataFrame(train, columns=[['Y', 'X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3']])
train

In [ ]:
X = train.drop(['Y'], axis=1)
y = train['Y']

In [ ]:
## 교차검증 시 데이터 분할 시각화 helper function 가져오기
from sklearn.model_selection import TimeSeriesSplit 
n_split = 4
tscv = TimeSeriesSplit(n_splits=n_split)
X = X.set_index(data["datetime"])

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
  print("Fold: {}".format(fold))
  print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
  print("\n")
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
def seed_everthing(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everthing(1)

In [ ]:
X_train

In [ ]:
model = Sequential()

# model.add(keras.Input(shape=(X_train.shape[1],1)))
# model.add(Dense(self.units, activation='tanh')
# model.layers.Embedding(67, 48, input_length = 1)
# model.add(layers.Embedding(input_dim = 67, output_dim = 1))
model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))

# RNN 첫번째 은닉층 추가
model.add(SimpleRNN(units=X_train.shape[1], activation='tanh', return_sequences = True))
model.add(Dropout(0.6)) # 노드간의 커넥트를 랜덤하게 끊어줌

# RNN 두번째 은닉층 추가
model.add(SimpleRNN(int(X_train.shape[1]/2), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.5))

# RNN 세번째 은닉층 추가
model.add(SimpleRNN(int(X_train.shape[1]/3), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.3))

# # RNN 네번째 은닉층 추가
# model.add(SimpleRNN(int(self.units/4), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
# model.add(Dropout(0.3))

model.add(SimpleRNN(10, activation='tanh', return_sequences = False)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.3))

# # RNN 세번째 은닉층 추가
# # model.add(SimpleRNN(self.units, activation='tanh', return_sequences=True))
# model.add(Dense(units = self.units / 4, activation="ReLU"))
# # model.add(SimpleRNN(units = 10, activation='tanh', return_sequences=True))
# model.add(Dropout(0.2))

# 출력층 추가: 분류모델 제외하고 활성화함수 사용X
model.add(Dense(units = 1)) # 다대일

## [3] compile: RNN모델을 학습시키기 위한 학습과정 설정하는 단계
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

## [4] 학습데이터로 RNN모델 학습
# earlystopping 선언
save_folder = '/content/drive/MyDrive/Colab Notebooks/교수님/hdf5/'
model_path = save_folder + 'mnist-' + '{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps = 100, decay_rate = 0.96, staircase=True)
scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

hist = model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=100,  batch_size =16 ,callbacks=[checkpoint, early_stopping, scheduler])

In [ ]:
y_vloss = hist.history['loss']
y_loss = hist.history['val_loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()